## Задание 1
### Необходимо написать функцию для подсчета retention.

In [16]:
import pandas as pd
import scipy.stats as st
import requests
from urllib.parse import urlencode

In [17]:
def retention():
    reg_data=pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-o-savicheva/shared/problem1-reg_data.csv', sep=';')
    auth_data=pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-o-savicheva/shared/problem1-auth_data.csv', sep=';')
    # переводим в формат datetime
    reg_data['reg_ts']=pd.to_datetime(reg_data.reg_ts, unit='s')
    auth_data['auth_ts']=pd.to_datetime(auth_data.auth_ts, unit='s')
    # объединяем в 1 dataframe 
    total_data=reg_data.merge(auth_data, on='uid')
    # присваиваем когорту по дню авторизации
    total_data['cohort_day']=total_data.groupby('uid').auth_ts.transform('min').dt.to_period('D')
    # рассчитываем номер периода
    total_data['cohort_period']=(total_data.auth_ts-total_data.reg_ts).dt.days
    # создаем сводную таблицу
    cohort_data=total_data.groupby(['cohort_day','cohort_period']).uid.nunique().reset_index()
    cogort_pivot=cohort_data.pivot(columns='cohort_period', index='cohort_day', values='uid').fillna(0)
    cohort_size=cogort_pivot.iloc[:,0]
    retention_matrix=cogort_pivot.divide(cohort_size, axis=0)
    return retention_matrix

In [18]:
retention()

cohort_period,0,1,2,3,4,5,6,7,8,9,...,7700,7703,7705,7708,7711,7715,7719,7720,7726,7728
cohort_day,,,,,,,,,,,,,,,,,,,,,
1998-11-18,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1999-07-22,1.0,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2000-01-13,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-05-28,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-09-16,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-19,1.0,0.037944,0.037332,0.047736,0.012852,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-09-20,1.0,0.043399,0.037286,0.011002,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-09-21,1.0,0.041514,0.007326,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Задание 2

#### Необходимо найти лучший набор акционных предложений и определить какие метрики стоит проанализировать для принятия правильного решения и как. Для этого даны результаты A/B теста, в котором двум группам пользователей предлагались различные наборы акционных предложений. Известно, что ARPU в тестовой группе выше на 5%, чем в контрольной. При этом в контрольной группе 1928 игроков из 202103 оказались платящими, а в тестовой – 1805 из 202667.

In [19]:
#Сохраняем файл с яндекс диска
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/SOkIsD5A8xlI7Q'  
final_url = base_url + urlencode({'public_key': public_key})
response = requests.get(final_url)

# Проверяем, успешен ли запрос
if response.status_code == 200:
    download_url = response.json().get('href')
    
    # Загружаем файл и сохраняем его
    download_response = requests.get(download_url)
    if download_response.status_code == 200:
        with open('downloaded_file.csv', 'wb') as f:
            f.write(download_response.content)
        print("Файл успешно загружен и сохранен как 'downloaded_file.csv'.")
    else:
        print(f"Ошибка при загрузке файла: {download_response.status_code} - {download_response.text}")
else:
    print(f"Ошибка при получении ссылки: {response.status_code} - {response.text}")

Файл успешно загружен и сохранен как 'downloaded_file.csv'.


In [20]:
задание_2=pd.read_csv('downloaded_file.csv', sep=';')

In [21]:
control=задание_2.query('testgroup=="a"')

In [22]:
test=задание_2.query('testgroup=="b"')

#### Для определения лучшего набора предложений необходимо проанализировать следующие метрики:  

- ARPU. Отражает среднюю выручку на пользователя.

- ARPPU. Отражает среднюю выручку на платящего пользователя.

- Конверсия в оплату. Она показывает как много пользователей совершает покупку из всех пользователей, учавствовавших в эксперименте.

ARPU в контрольной группе:

In [23]:
(control.revenue.sum()/control.shape[0]).round(2)

25.41

ARPU в тестовой группе:

In [24]:
(test.revenue.sum()/test.shape[0]).round(2)

26.75

In [25]:
((test.revenue.sum()/test.shape[0]).round(2) / (control.revenue.sum()/control.shape[0]).round(2) - 1).round(2)

0.05

ARPU в тестовой группе выше на 5%, чем в контрольной. Определим, есть ли статистически значимая разница. Выборка достаточно большая, сравниваем средние, поэтому применим t-test.

In [26]:
st.ttest_ind(test.revenue,control.revenue)

Ttest_indResult(statistic=0.6242026493616787, pvalue=0.532494858971837)

P-value > 0.05, поэтому мы можем говорить об отсутствии статистически значимых различий в средней выручке на пользователя.

#### Рассчитаем ARPPU:

ARPPU в контрольной группе:

In [27]:
control.revenue.sum()/control.query('revenue!=0').shape[0]

2663.9984439834025

ARPPU в тестовой группе:

In [28]:
test.revenue.sum() / test.query('revenue!=0').shape[0]

3003.6581717451522

In [29]:
st.ttest_ind(test.query('revenue > 0').revenue, control.query('revenue > 0').revenue)

Ttest_indResult(statistic=1.5917100176862002, pvalue=0.11153459157259504)

ARPPU в тестовой группе выше, чем в контрольной, но и платящих пользователей в тестовой группе меньше, p-value > 0.05, статистически значимые различия в средней выручке на платящего пользователя отсутствуют.

#### Рассчитаем конверсию в оплату

In [30]:
задание_2['pay']=задание_2.revenue.apply(lambda x: 'yes' if x!=0 else 'no')

Конверсия в оплату в контрольной группе:

In [31]:
задание_2.query('testgroup=="a"').pay.value_counts(normalize=True).mul(100).round(2)

no     99.05
yes     0.95
Name: pay, dtype: float64

Конверсия в оплату в тестовой группе:

In [32]:
задание_2.query('testgroup=="b"').pay.value_counts(normalize=True).mul(100).round(2)

no     99.11
yes     0.89
Name: pay, dtype: float64

Для определения статистически значимой разницы конверсии в оплату необходимо применить статистический критерий Хи-квадрат, так как у нас конверсия.  

Н0: распределение оплат по группам равномерное  
Н1: распределение неравномерное

In [35]:
pd.crosstab(задание_2.pay, задание_2.testgroup)

testgroup,a,b
pay,,
no,200175,200862
yes,1928,1805


In [36]:
st.chi2_contingency(pd.crosstab(задание_2.testgroup, задание_2.pay))

(4.374729521260405,
 0.03647561892312613,
 1,
 array([[200239.10075104,   1863.89924896],
        [200797.89924896,   1869.10075104]]))

##### Вывод:
   P-value=0.036, что меньше уровня значимости 0.05. Это значит, что в распределении оплат имеются различия по группам. В ARPU B ARPPU двух групп статистически значимых различий нет, а конверсия в контрольной группе выше, можем говорить о том, что набор предложений  в контрольной группе можно считать лучшим.

## Задание 3

#### Необходимо определить, с помощью каких метрик можно оценить результаты последнего прошедшего тематического события, в которых игроки могут получить уникальные предметы для сада и персонажей, дополнительные монеты или бонусы. Для получения награды требуется пройти ряд уровней за определенное время.
#### Есть еще другое событие: нужно определить набор метрик, при условии, что при каждой неудачной попытке выполнения уровня игрок будет откатываться на несколько уровней назад. Изменится ли набор метрик оценки результата? Если да, то как?

### Метрики, с помощью которых можно оценить результаты последнего прошедшего события:
#### Метрики роста:
- DAU - количество уникальных пользователей за сутки (вырастет при положительном результате)
- Количество платящих пользователей. Если в игре за прохождение некоторых уровней необходимо платить, то количество платящих пользователей увеличится, что говорит о положительном результате прошедшего события.

#### Метрики продукта:
- Время сессии пользователя в игре. Увеличение длительности сессии ведет к росту вовлеченности пользователя и долгосрочным бизнес-результам.
- Retention. Увеличится при положительном результате прошедшего события. Вернутся пользователи, которые не пользовались продуктом какое-то время, для того, чтобы получить награды, что также положительно повлияет на бизнес-результат.

### Если при каждой неудачной попытке выполнения уровня игрок будет откатываться на несколько уровней назад, то набор метрик будет следующим:
#### Метрики роста:
- DAU
- Количество платящих пользователей

#### Метрики продукта:
- Время сессии пользователя в игре
- Конверсия в пользователя, не прошедшего уровень и следовательно откатившегося на несколько уровней назад (если таких пользователей много, то событие может привести к оттоку пользователей)
- Churn rate - % оттока за определенное время (чтобы понять не спугнули ли пользователей такие условия события)